In [1]:
import os

import numpy as np

from keras.models import load_model

import cv2

import pandas as pd

import time

import loss_functions


Using TensorFlow backend.


In [20]:
model_path = '../models/trained/unet_101_binary_crossentropy_adam_accuracy.h5'
model_input_size = 101 if '101' in model_path else 128

test_img_path = '../data/test/'
valid_img_path = '../data/{}/all_samples/'.format(model_input_size)
submission_path = '../submissions/'+ model_path.split('/')[-1][:-3] + '.csv'

need_resize = model_input_size != 101

In [16]:
for root, dirs, test_img_names in os.walk(test_img_path):
    test_img_ids = [img_name[:-4] for img_name in test_img_names]
    test_img_paths = [test_img_path + img_name for img_name in test_img_names]

In [ ]:
x_valid = np.load(valid_img_path + 'x_valid.npy')
y_valid = np.load(valid_img_path + 'y_valid.npy')

In [21]:
model = load_model(model_path, custom_objects={'iou':loss_functions.iou})

W0812 17:46:02.152350 15060 deprecation.py:506] From c:\users\robert.colt\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
def rle_encode(im):
    pixels = im.flatten(order = 'F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
#     print(runs, runs[1::2])
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)



In [ ]:
# find best threshhod
thresh_iou = {}
for thresh in np.arange(0.55, 1.0, 0.05):
    avg_iou = 0
    for i, img in enumerate(range(x_valid.shape[0])):
        prediction = model.predict(x_valid[i].reshape(-1, 101, 101, 1))
        avg_iou += loss_functions.iou_np((prediction > thresh).astype(int), y_valid[i])
    
    avg_iou = avg_iou / x_valid.shape[0]
    thresh_iou[thresh] = avg_iou
    print(thresh_iou)

In [22]:
rle_masks = []
maxcnt = 10
start = time.time()


for i, img_path in enumerate(test_img_paths):
    if i%200 == 0:
        print('{}'.format(i))
    img = cv2.imread(img_path, 0)/255
    if i == 0:
        print(img.mean())
    if need_resize:
        img = cv2.resize(img, (model_input_size , model_input_size), interpolation=cv2.INTER_CUBIC)
    img = img.reshape((1, model_input_size, model_input_size, 1))
    prediction = model.predict(img).reshape((model_input_size, model_input_size, 1))
    prediction = (prediction > 0.7).astype(float)
    if need_resize:
        prediction = cv2.resize(prediction, (101,101), interpolation=cv2.INTER_AREA)
        prediction = prediction >= 0.99
    rle_masks.append(rle_encode(prediction))

print('finished 0.89')
submission = {'id':test_img_ids, 'rle_mask':rle_masks}

submission_df = pd.DataFrame.from_dict(submission)
submission_df.to_csv(submission_path, index=0)

0
0.541629328920079
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
10000
10200
10400
10600
10800
11000
11200
11400
11600
11800
12000
12200
12400
12600
12800
13000
13200
13400
13600
13800
14000
14200
14400
14600
14800
15000
15200
15400
15600
15800
16000
16200
16400
16600
16800
17000
17200
17400
17600
17800
finished 0.89


In [17]:
submission = {'id':test_img_ids, 'rle_mask':rle_masks}

submission_df = pd.DataFrame.from_dict(submission)
submission_df.to_csv(submission_path, index=0)

In [23]:
submission_df.head()

,id,rle_mask
0,0005bb9630,
1,000a68e46c,
2,000c8dfb2a,
3,000d0a5f6c,
4,001ef8fc87,1 7358 7363 7 7372 79 7475 71 7576 64 7677 58 ...
